### Training Sentence Transformer Model

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7dfaa721ca098017803eb06656b775a929f2c13e45adab27797b8371d3100fc8
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [2]:
# from sentence_transformers import SentenceTransformer, models

# ## Step 1: use an existing language model
# word_embedding_model = models.Transformer('distilroberta-base')

# ## Step 2: use a pool function over the token embeddings
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# ## Join steps 1 and 2 using the modules argument
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [3]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_id)

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.4 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

#dataset_id = "embedding-data/QQP_triplets"
#dataset = load_dataset(dataset_id)

In [6]:

dataset_id = "snli"
dataset = load_dataset(dataset_id)

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print(f"- The {dataset_id} dataset has {dataset['train'].num_rows} examples.")
#print(f"- Each example is a {type(dataset['train'][0])} with a {type(dataset['train'][0]['set'])} as value.")
print(f"- Examples look like this: {dataset['train'][0]}")

- The snli dataset has 550152 examples.
- Examples look like this: {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


In [15]:
from sentence_transformers import InputExample

train_examples = []
train_data = dataset['train']
# For agility we only 1/2 of our available data
n_examples = dataset['train'].num_rows // 2

### We found some sample has -1 values during training so we replace them with 3 as per our loss is ge 0
for i in range(n_examples):
  example = train_data[i]
  if example['label'] == -1:
    label_sel = 3
  else:
    label_sel = example['label']
  train_examples.append(InputExample(texts=[example['premise'], example['hypothesis']],label=label_sel))

In [16]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 275076 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [18]:
### Loss functions for training a Sentence Transformers model
from sentence_transformers import losses

train_loss = losses.SoftmaxLoss(model=model,sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=4)

In [19]:
### Train the model
num_epochs = 2

warmup_steps = int(len(train_dataloader) * num_epochs * 0.01) #10% of train data

In [20]:
train_loss

SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  )
  (classifier): Linear(in_features=1152, out_features=4, bias=True)
  (loss_fct): CrossEntropyLoss()
)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

References : 

1. https://colab.research.google.com/github/huggingface/blog/blob/main/notebooks/95_Training_Sentence_Transformers.ipynb#scrollTo=0HeVHzC-8bbe

2. https://huggingface.co/blog/how-to-train-sentence-transformers

3. https://huggingface.co/datasets/snli

4. https://www.sbert.net/docs/package_reference/losses.html#softmaxloss